In [3]:
from fastapi import FastAPI, UploadFile, File, HTTPException
import pandas as pd
from io import BytesIO

app = FastAPI(title="AI Email Agent API")

# Your existing CSV loader function (adapted)
def load_csv_file(file) -> pd.DataFrame:
    try:
        df = pd.read_csv(file)
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Error reading CSV: {e}")

    df.columns = [c.strip().lower() for c in df.columns]

    if "email" not in df.columns:
        raise HTTPException(status_code=400, detail="CSV is missing required column: 'email'")

    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].astype(str).str.strip()

    df = df[df["email"].notna() & (df["email"] != "")]
    df = df.drop_duplicates(subset=["email"])

    return df

# FastAPI endpoint
@app.post("/upload_csv")
async def upload_csv(file: UploadFile = File(...)):
    if not file.filename.endswith(".csv"):
        raise HTTPException(status_code=400, detail="File must be a CSV.")

    contents = await file.read()
    df = load_csv_file(BytesIO(contents))

    preview = df.head().to_dict(orient="records")
    total_rows = len(df)

    return {"preview": preview, "total_rows": total_rows}


In [ ]:
# Function to validate DataFrame rows (reusing the previous logic)
def validate_recipients_df(df: pd.DataFrame):
    valid_rows = []
    invalid_rows = []

    for idx, row in df.iterrows():
        try:
            recipient = RecipientRow(**row.to_dict())
            valid_rows.append(recipient.dict())  # convert to dict for JSON response
        except ValidationError as e:
            invalid_rows.append({
                "row_index": idx,
                "errors": e.errors(),
                "data": row.to_dict()
            })

    return valid_rows, invalid_rows

# FastAPI endpoint to validate uploaded CSV
@app.post("/validate_recipients")
async def validate_recipients(file: UploadFile = File(...)):
    if not file.filename.endswith(".csv"):
        raise HTTPException(status_code=400, detail="File must be a CSV.")

    contents = await file.read()
    df = load_csv_file(BytesIO(contents))  # reuse CSV loader from previous endpoint

    valid_rows, invalid_rows = validate_recipients_df(df)

    return {
        "valid_count": len(valid_rows),
        "invalid_count": len(invalid_rows),
        "valid_rows": valid_rows,
        "invalid_rows": invalid_rows
    }
